In [2]:
import sys
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ExponentialLR, StepLR
import torch.nn.functional as F
use_cuda = torch.cuda.is_available()
import numpy as np
from tqdm import tqdm, trange
import pickle
from rdkit import Chem, DataStructs
from stackRNN import StackAugmentedRNN
from data import GeneratorData
from utils import canonical_smiles, time_since
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import seaborn as sns
import re
import pandas as pd
from pymatgen import Composition, Element
from collections import Counter
import json
import time
# from keras.preprocessing import sequence
from itertools import groupby
from sklearn.preprocessing import StandardScaler
from reinforcement import Reinforcement
from data import PredictorData
from utils import get_desc, get_fp
from roost_models.roost_model import predict_formation_energy, predict_bulk_mod, predict_shear_mod, predict_band_gap
from constraints.checkers import check_neutrality, check_neutrality, check_neutrality_multiple, check_electronegativity_multiple, check_both_multiple
from metrics import similarity_to_nearest_neighbor
my_predictor = None

In [3]:
with open('/home/jupyter/RL_paper/element_sets/roost_unique_elem_dict.pkl', 'rb') as f:
    roost_elem_dict = pickle.load(f)

In [4]:
print(roost_elem_dict)

{'mp-form-enthalpy.csv': ['La', 'Rh', 'Mg', 'Bi', 'B', 'Lu', 'Cu', 'S', 'Yb', 'Ni', 'H', 'Ce', 'Se', 'Pr', 'Cr', 'Sc', 'Ga', 'Fe', 'Ti', 'V', 'Ca', 'Zr', 'N', 'Nb', 'Co', 'Si', 'Y', 'C', 'Nd', 'Ru', 'Sm', 'Mn', 'O', 'K', 'Rb', 'Er', 'Pt', 'Ba', 'Hg', 'Sb', 'In', 'Pd', 'Cd', 'Na', 'Au', 'Li', 'I', 'Hf', 'Al', 'Mo', 'Be', 'Sn', 'Tm', 'Tl', 'Pb', 'Cl', 'F', 'Ag', 'Tb', 'Cs', 'Ta', 'P', 'Br', 'Eu', 'Zn', 'Pa', 'Ac', 'Tc', 'Th', 'Re', 'U', 'Ho', 'As', 'Os', 'Dy', 'Sr', 'Ge', 'W', 'Te', 'Ir', 'Np', 'Pu', 'Gd', 'Pm', 'Xe'], 'mp-band-gap.csv': ['Ac', 'O', 'S', 'Ag', 'Te', 'Al', 'B', 'Br', 'Cl', 'Cr', 'F', 'Fe', 'Ga', 'H', 'I', 'In', 'Mg', 'N', 'Sc', 'Tl', 'Zn', 'Au', 'C', 'Sn', 'Se', 'As', 'P', 'Hg', 'Mo', 'Bi', 'Sb', 'Ge', 'Pb', 'Pd', 'W', 'Xe', 'Pt', 'Rh', 'Si', 'Cd', 'V', 'Ru', 'Co', 'Cu', 'Ni', 'Os', 'Tc', 'Ir', 'Re', 'Kr', 'Ba', 'Be', 'Dy', 'Er', 'Gd', 'Ho', 'La', 'Lu', 'Mn', 'Na', 'Nb', 'Nd', 'Pr', 'Sm', 'Ta', 'Tb', 'Tm', 'Y', 'Yb', 'Li', 'Sr', 'Ca', 'U', 'Zr', 'Ti', 'Ce', 'Eu', 'Hf', 'P

In [5]:
for key in roost_elem_dict:
    print(len(roost_elem_dict[key]))

85
86
84
84


In [7]:
gen_data_path = '/home/jupyter/CJK/RL/data/mp_all_data_cleaned_oxides_csv.csv'

In [8]:
df = pd.read_csv(gen_data_path)

In [9]:
df

,Unnamed: 0,pretty_formula,pretty_formula_coeff,full_formula,energy,energy_per_atom,volume,formation_energy_per_atom,nsites,unit_cell_formula,...,icsd_ids,cif,total_magnetization,material_id,oxide_type,tags,elasticity,piezo,diel,deprecated
0,0,Sm2Mn2O5,Sm2Mn2O5,Sm4Mn4O10,-143.453743,-7.969652,255.573521,-2.888540,18,"{'Sm': 4.0, 'Mn': 4.0, 'O': 10.0}",...,[],# generated using pymatgen\ndata_Sm2Mn2O5\n_sy...,9.999539,mp-1076289,oxide,[],NaN,NaN,NaN,False
1,1,KCrO3,K1Cr1O3,K1Cr1O3,-29.926341,-5.985268,57.724275,-1.675802,5,"{'K': 1.0, 'Cr': 1.0, 'O': 3.0}",...,[],# generated using pymatgen\ndata_KCrO3\n_symme...,1.000021,mp-1076732,oxide,[],NaN,NaN,NaN,False
2,2,Pd2PbO4,Pd2Pb1O4,Pd4Pb2O8,-77.281682,-5.520120,180.950010,-1.074941,14,"{'Pd': 4.0, 'Pb': 2.0, 'O': 8.0}",...,"[263049, 263050]",# generated using pymatgen\ndata_Pd2PbO4\n_sym...,0.000091,mp-1103838,oxide,['Lead dipalladium tetraoxide'],NaN,NaN,NaN,False
3,3,LiSbP(OF3)2,Li1Sb1P1O2F6,Li2Sb2P2O4F12,-117.365628,-5.334801,364.251526,-2.728436,22,"{'Li': 2.0, 'Sb': 2.0, 'P': 2.0, 'O': 4.0, 'F'...",...,[],# generated using pymatgen\ndata_LiSbP(OF3)2\n...,0.001084,mp-1176602,oxide,[],NaN,NaN,NaN,False
4,4,NaH2BrO5,Na1H2Br1O5,Na4H8Br4O20,-153.403082,-4.261197,630.109037,-0.811228,36,"{'Na': 4.0, 'H': 8.0, 'Br': 4.0, 'O': 20.0}",...,[],# generated using pymatgen\ndata_NaH2BrO5\n_sy...,0.000214,mp-1179975,hydroxide,[],NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22550,22550,AlMoO3,Al1Mo1O3,Al4Mo4O12,-145.622759,-7.281138,222.552713,-2.446718,20,"{'Al': 4.0, 'Mo': 4.0, 'O': 12.0}",...,[],# generated using pymatgen\ndata_AlMoO3\n_symm...,2.999963,mvc-15905,oxide,[],NaN,NaN,NaN,False
22551,22551,Zn2Cu2O5,Zn2Cu2O5,Zn4Cu4O10,-82.377417,-4.576523,194.067588,-1.018445,18,"{'Zn': 4.0, 'Cu': 4.0, 'O': 10.0}",...,[],# generated using pymatgen\ndata_Zn2Cu2O5\n_sy...,0.007697,mvc-3812,oxide,[],NaN,NaN,NaN,False
22552,22552,CaSnO2,Ca1Sn1O2,Ca4Sn4O8,-95.390088,-5.961881,269.869783,-2.327607,16,"{'Ca': 4.0, 'Sn': 4.0, 'O': 8.0}",...,[],# generated using pymatgen\ndata_CaSnO2\n_symm...,0.000054,mvc-4868,oxide,[],NaN,NaN,NaN,False
22553,22553,BaCa(AgO2)4,Ba1Ca1Ag4O8,Ba1Ca1Ag4O8,-62.275844,-4.448275,231.897662,-0.923812,14,"{'Ba': 1.0, 'Ca': 1.0, 'Ag': 4.0, 'O': 8.0}",...,[],# generated using pymatgen\ndata_BaCa(AgO2)4\n...,0.013947,mvc-545,oxide,[],NaN,NaN,NaN,False


In [10]:
# restrict elements to that of a particular task we are interested in
restricted_targets = []
for target in df['pretty_formula_coeff']:
    coeffs = list(Composition(target).get_el_amt_dict().values())
    # limit to integer coeffs and 5 elements max
    if any(x >= 10 for x in coeffs) or len(coeffs) > 5:
        continue
    els = [el.symbol for el in Composition(target).elements]
    for el in els:
        if el not in roost_elem_dict['mp-bulk-mod.csv']:
            print(el)
    if all(el in roost_elem_dict['mp-bulk-mod.csv'] for el in els) and el not in ['Xe', 'Kr']:
        restricted_targets.append(target)


In [11]:
print(len(restricted_targets))

22186


In [12]:
all_elements = roost_elem_dict['mp-bulk-mod.csv']
all_numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# create combination of all elems + all numbers to choose from
all_elements_numbers = []
for element in all_elements:
    for number in all_numbers:
        all_elements_numbers.append(element+number)

In [13]:
# <, > are starting and ending characters
# tokens = ['<', '>'] + all_elements + all_numbers
tokens = ['<', '>'] + all_elements_numbers

In [14]:
gen_data = GeneratorData(training_data_path=gen_data_path, max_len = 5,
                         cols_to_read=[2], keep_header=False, tokens=tokens)

In [15]:
gen_data.all_characters

['<',
 '>',
 'Yb0',
 'Yb1',
 'Yb2',
 'Yb3',
 'Yb4',
 'Yb5',
 'Yb6',
 'Yb7',
 'Yb8',
 'Yb9',
 'Ni0',
 'Ni1',
 'Ni2',
 'Ni3',
 'Ni4',
 'Ni5',
 'Ni6',
 'Ni7',
 'Ni8',
 'Ni9',
 'H0',
 'H1',
 'H2',
 'H3',
 'H4',
 'H5',
 'H6',
 'H7',
 'H8',
 'H9',
 'Sc0',
 'Sc1',
 'Sc2',
 'Sc3',
 'Sc4',
 'Sc5',
 'Sc6',
 'Sc7',
 'Sc8',
 'Sc9',
 'Ga0',
 'Ga1',
 'Ga2',
 'Ga3',
 'Ga4',
 'Ga5',
 'Ga6',
 'Ga7',
 'Ga8',
 'Ga9',
 'Fe0',
 'Fe1',
 'Fe2',
 'Fe3',
 'Fe4',
 'Fe5',
 'Fe6',
 'Fe7',
 'Fe8',
 'Fe9',
 'Ca0',
 'Ca1',
 'Ca2',
 'Ca3',
 'Ca4',
 'Ca5',
 'Ca6',
 'Ca7',
 'Ca8',
 'Ca9',
 'Zr0',
 'Zr1',
 'Zr2',
 'Zr3',
 'Zr4',
 'Zr5',
 'Zr6',
 'Zr7',
 'Zr8',
 'Zr9',
 'N0',
 'N1',
 'N2',
 'N3',
 'N4',
 'N5',
 'N6',
 'N7',
 'N8',
 'N9',
 'Nb0',
 'Nb1',
 'Nb2',
 'Nb3',
 'Nb4',
 'Nb5',
 'Nb6',
 'Nb7',
 'Nb8',
 'Nb9',
 'Co0',
 'Co1',
 'Co2',
 'Co3',
 'Co4',
 'Co5',
 'Co6',
 'Co7',
 'Co8',
 'Co9',
 'Si0',
 'Si1',
 'Si2',
 'Si3',
 'Si4',
 'Si5',
 'Si6',
 'Si7',
 'Si8',
 'Si9',
 'Y0',
 'Y1',
 'Y2',
 'Y3',
 'Y4',
 'Y5',
 'Y6',

In [16]:
print(len(gen_data.file))

22186


In [17]:
# check if gen_data and restricted_targets are the same
for i in range(len(restricted_targets)):
    if restricted_targets[i] != gen_data.file[i][1:-1]:
        print(restricted_targets[i], gen_data.file[i])

In [18]:
print(gen_data.file[5000])
print(gen_data.char_tensor(gen_data.file[5000]))

<Ba1Pr1Cu2O5>
tensor([  0, 313, 513, 294, 217,   1], device='cuda:0')


In [19]:
hidden_size = 1500
stack_width = 1500
stack_depth = 10
layer_type = 'GRU'
lr = 0.001
optimizer_instance = torch.optim.Adadelta

my_generator = StackAugmentedRNN(input_size=gen_data.n_characters, hidden_size=hidden_size,
                                 output_size=gen_data.n_characters, layer_type=layer_type,
                                 n_layers=3, is_bidirectional=True, has_stack=True,
                                 stack_width=stack_width, stack_depth=stack_depth, 
                                 use_cuda=use_cuda, 
                                 optimizer_instance=optimizer_instance, lr=lr)

In [20]:
def plot_hist(prediction, n_to_generate):
    print("Mean value of predictions:", prediction.mean())
    print("Proportion of valid compounds:", len(prediction)/n_to_generate)
    ax = sns.kdeplot(prediction, shade=True)
    ax.set(xlabel='Predicted formation energy', 
           title='Distribution of predicted formation energy for generated compounds')
    plt.show()

In [21]:
def estimate_and_update(generator, predictor, n_to_generate):
    generated = []
    pbar = tqdm(range(n_to_generate))
    pbar.set_description("Generating molecules...")
    for i in pbar:
        generated_string = generator.evaluate(gen_data, predict_len=5)
        # remove all starting '<' and ending '>' from generated_strings
        # first char always '<'
        generated_string = generated_string[1:]
        if len(generated_string) > 0:
            if generated_string[-1] == '>':
                # terminated successfully
                generated.append(generated_string[:-1])
            else:
                # got cut off at max length steps
                generated.append(generated_string)

    valid_smiles = []
    prediction = []
    
    pbar = tqdm(generated, total=len(generated), position=0, leave=True)
    for sm in pbar:
        pbar.set_description("Generating predictions...")
        # predict
        try:
            c = Composition(sm)
            form_e = predict_formation_energy(sm)
            prediction.append(form_e)
            valid_smiles.append(sm)
        except:
            print(f"Invalid compound: {sm}")
            continue
            
    prediction = np.array(prediction)                     
    plot_hist(prediction, n_to_generate)
        
    return valid_smiles, prediction

In [22]:
def fit(my_generator, data, n_iterations, all_losses=[], print_every=100,
        plot_every=10, augment=False):
    """
    This methods fits the parameters of the model. Training is performed to
    minimize the cross-entropy loss when predicting the next character
    given the prefix.

    Parameters
    ----------
    data: object of type GeneratorData
        stores information about the generator data format such alphabet, etc

    n_iterations: int
        how many iterations of training will be performed

    all_losses: list (default [])
        list to store the values of the loss function

    print_every: int (default 100)
        feedback will be printed to std_out once every print_every
        iterations of training

    plot_every: int (default 10)
        value of the loss function will be appended to all_losses once every
        plot_every iterations of training

    augment: bool (default False)
        parameter specifying if SMILES enumeration will be used. For mode
        details on SMILES enumeration see https://arxiv.org/abs/1703.07076

    Returns
    -------
    all_losses: list
        list that stores the values of the loss function (learning curve)
    """
    start = time.time()
    loss_avg = 0

    if augment:
        smiles_augmentation = SmilesEnumerator()
    else:
        smiles_augmentation = None

    for epoch in trange(1, n_iterations + 1, desc='Training in progress...'):
        # save model every 1000 epochs
        if epoch % 1000 == 0:
            my_generator.save_model(f'/home/ReLeaSE/checkpoints/paper/unbiased/unbiased_{epoch}')
            final_compounds, form_e_trained = estimate_and_update(my_generator, None, n_to_generate=1000)
            # save statistics

            # Charge neutrality and electronegativity
            cn_trained_good, cn_trained_bad = check_neutrality_multiple(final_compounds)
            en_trained_good, en_trained_bad = check_electronegativity_multiple(final_compounds)
            # Uniqueness
            final_compounds_unique = list(set(final_compounds))
            # EMD
            final_compounds_EMD_mean, final_compounds_EMD_std = similarity_to_nearest_neighbor(final_compounds)

            results = {}
            results['iteration'] = epoch
            results['validity'] = len(final_compounds) / 1000
            results['form_e_trained_mean'] = np.mean(form_e_trained)
            results['form_e_trained_std'] = np.std(form_e_trained)
            results['charge_neutral_trained'] = len(cn_trained_good)/(len(cn_trained_good)+len(cn_trained_bad))
            results['electronegativity_balanced_trained'] = len(en_trained_good)/(len(en_trained_good)+len(en_trained_bad))
            results['unique_trained'] = len(final_compounds_unique)/len(final_compounds)
            results['final_compounds_EMD_mean'], results['final_compounds_EMD_std'] = final_compounds_EMD_mean, final_compounds_EMD_std
            results['all_losses'] = all_losses

            print(results)

            with open(f'/home/jupyter/CJK/RL/paper/data/unbiased/unbiased_{epoch}_results.pkl', 'wb') as f:
                pickle.dump(results, f)
                
        inp, target = data.random_training_set(smiles_augmentation)
        loss = my_generator.train_step(inp, target)
        loss_avg += loss

        if epoch % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch,
                                           epoch / n_iterations * 100, loss)
                  )
            print(my_generator.evaluate(data=data, prime_str = '<',
                                predict_len=5), '\n')

        if epoch % plot_every == 0:
            all_losses.append(loss_avg / plot_every)
            loss_avg = 0
    return all_losses

In [ ]:
# losses = my_generator.fit(data=gen_data, n_iterations=10000, print_every=1000)
fit(my_generator=my_generator, data=gen_data, n_iterations=20000, print_every=1000)

In [ ]:
plt.plot(losses)